## set the var env and the global var

In [49]:
%reset -f
import os
from dotenv import load_dotenv
from datetime import datetime

# Get the SEO keywords and parent folder as input from the user
seo_keywords = "The Moroccan Moussem of Roses: Valley Festival" 

# 1: moroccoheritage 2 : gobitcode
website = "1"
current_date = datetime.now().strftime('%Y-%m-%d')
# app requirements


In [50]:
if website == '1':
    file_path = f'./format/moroccoheritage.mdx'
    blog_path = "/home/adil/repo/morocco-heritage/data/blog/"
    image_path = "/home/adil/repo/morocco-heritage/public/static/images"
else:
    file_path = f'./format/gobitcode.mdx'
    blog_path = "/home/adil/repo/gobitcode/data/blog/"
    image_path = "/home/adil/repo/gobitcode/public/static/images"

with open(file_path, 'r') as file:
    mdx_format = file.read()


mdx_format = mdx_format.replace("current_date", current_date)

# Load environment variables from .env file
load_dotenv()

# Get the API key from the environment variable
groq_api_key = os.getenv('GROQ_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
google_cx = os.getenv('GOOGLE_CX')

print(mdx_format)


---
title: The Timeless Elegance of the Moroccan Djellaba for Men
date: '2025-04-08'  
tags: ['moroccan fashion', 'traditional clothing', 'djellaba']
draft: false
summary: Explore the cultural significance and enduring appeal of the Moroccan Djellaba for men, a garment that represents both tradition and style in Morocco.
---

## Introduction

Brief introduction about the **Moroccan Djellaba** and its cultural importance.

<TOCInline toc={props.toc} exclude="Introduction" />

## What is the Moroccan Djellaba?

Explanation of the **djellaba**, its materials, and uses.

<img src="/static/images/image1.png" alt="Moroccan Djellaba" width="400" height="300" />

## A Brief History of the Djellaba

Summary of the **djellaba**'s origins and evolution over time.

## Types of Moroccan Djellabas for Men

- **Winter Djellaba**
- **Summer Djellaba**
- **Ceremonial Djellaba**

<img src="/static/images/image2.png" alt="Modern Moroccan Djellaba" width="500" height="300" />

## Craftsmanship Behind the 

## ai generate the blog

In [51]:
from groq import Groq

client = Groq(api_key=groq_api_key)

completion = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {
            "role": "system",
            "content": f"You are a blog writer who writes blogs in this format: \n {mdx_format} \n Do not include any introductory text."
        },
        {
            "role": "user",
            "content": f"Write a long-form blog about '{seo_keywords}' \n Do not forget image links and alt text."
        }
    ],
    temperature=1,
    max_tokens=1024*4,
    top_p=1,
    stream=False,  # Set stream to False to get the response as one
    stop=None,
)

mdx_blog = completion.choices[0].message.content

print(mdx_blog)

---
title: The Moroccan Moussem of Roses: Valley Festival
date: '2025-04-08'  
tags: ['moroccan festivals', 'moussem of roses', 'valley festival']
draft: false
summary: Discover the sweet charm of the Moroccan Moussem of Roses, a vibrant valley festival that celebrates the beauty of nature and local traditions.
---

## Introduction

In the picturesque valleys of Morocco, a fragrant and colorful festival takes place every year, captivating the hearts of locals and tourists alike. The Moroccan Moussem of Roses, also known as the Valley Festival, is a unique celebration that honors the beauty of nature, local traditions, and the iconic rose. In this blog, we'll delve into the fascinating world of this festival, exploring its history, significance, and the unforgettable experiences it offers.

<TOCInline toc={props.toc} exclude="Introduction" />

## What is the Moussem of Roses?

The Moussem of Roses, also referred to as the Festival of Roses, is an annual celebration held in the Dades Val

## slugify

In [52]:
import re
# Function to generate a slug from the title
def generate_slug(title):
    # Convert to lowercase
    slug = title.lower()
    # Replace spaces and special characters with hyphens
    slug = re.sub(r'[^a-z0-9]+', '-', slug)
    # Remove leading and trailing hyphens
    slug = slug.strip('-')
    return slug


# Regular expression to extract the title
title_match = re.search(r'^title:\s*(.*)$', mdx_blog, re.MULTILINE)

if title_match:
    title = title_match.group(1)
    slug = generate_slug(title)
else:
    raise ValueError("Title not found in the blog")

parent_folder = generate_slug(seo_keywords)

print(f"Slug: {slug}")



Slug: the-moroccan-moussem-of-roses-valley-festival


## download images

In [53]:
from google_images_search import GoogleImagesSearch

def download_images(keyword,path,  limit=1):
    # You need to get your own API key and CX from Google Custom Search JSON API
    gis = GoogleImagesSearch(google_api_key,google_cx)

    search_params = {
        'q': keyword,
        'num': limit,
        'safe': 'off',
     #   'fileType': 'jpg|png',
       # 'imgType': 'photo',
      #  'imgSize': 'LARGE',  # Use a single size or remove this line if not needed
        # 'imgDominantColor': 'black'  # Use a single color or remove this line if not needed
    }

    gis.search(search_params=search_params)
    image = gis.results()[0]
    image.download(path)
    old_image_name = os.path.basename(image.path)
    # Split filename and extension
    filename, extension = os.path.splitext(old_image_name)
    # Generate new slugified name
    new_image_name = f"{generate_slug(filename)}{extension}"
    os.rename(image.path, os.path.join(path, new_image_name))
    return new_image_name





## download images

In [54]:



# Regex to match the <img> tag and capture the alt attribute
img_tag_regex = r'<img[^>]*alt="([^"]*)"[^>]*>'

# Find all matches
matches = re.findall(img_tag_regex, mdx_blog)



# Print the alt attributes
for match in matches:
    image_name =  download_images(f"{seo_keywords} {match}"  , os.path.join(image_path,parent_folder))
    print(f"alt attribute: {match}")
    new_src = f'/static/images/{parent_folder}/{image_name}'
    mdx_blog = re.sub(rf'(<img[^>]*src=")[^"]*("[^>]*alt="{re.escape(match)}"[^>]*>)', rf'\1{new_src}\2', mdx_blog)
    
print(mdx_blog) 
    

alt attribute: Moussem of Roses Valley
alt attribute: Moroccan Rose Farm
alt attribute: Moroccan Food
alt attribute: Gnawa Musicians
alt attribute: Souk Marketplace
---
title: The Moroccan Moussem of Roses: Valley Festival
date: '2025-04-08'  
tags: ['moroccan festivals', 'moussem of roses', 'valley festival']
draft: false
summary: Discover the sweet charm of the Moroccan Moussem of Roses, a vibrant valley festival that celebrates the beauty of nature and local traditions.
---

## Introduction

In the picturesque valleys of Morocco, a fragrant and colorful festival takes place every year, captivating the hearts of locals and tourists alike. The Moroccan Moussem of Roses, also known as the Valley Festival, is a unique celebration that honors the beauty of nature, local traditions, and the iconic rose. In this blog, we'll delve into the fascinating world of this festival, exploring its history, significance, and the unforgettable experiences it offers.

<TOCInline toc={props.toc} exclude

## correct title

In [55]:



# Regex to match and remove the colon from the title line
title_regex = r'^(title:.*?):(.*)$'

# Remove the colon from the title line
mdx_blog = re.sub(title_regex, r'\1\2', mdx_blog, flags=re.MULTILINE)

print(mdx_blog)

---
title: The Moroccan Moussem of Roses Valley Festival
date: '2025-04-08'  
tags: ['moroccan festivals', 'moussem of roses', 'valley festival']
draft: false
summary: Discover the sweet charm of the Moroccan Moussem of Roses, a vibrant valley festival that celebrates the beauty of nature and local traditions.
---

## Introduction

In the picturesque valleys of Morocco, a fragrant and colorful festival takes place every year, captivating the hearts of locals and tourists alike. The Moroccan Moussem of Roses, also known as the Valley Festival, is a unique celebration that honors the beauty of nature, local traditions, and the iconic rose. In this blog, we'll delve into the fascinating world of this festival, exploring its history, significance, and the unforgettable experiences it offers.

<TOCInline toc={props.toc} exclude="Introduction" />

## What is the Moussem of Roses?

The Moussem of Roses, also referred to as the Festival of Roses, is an annual celebration held in the Dades Vall

## save the blog

In [56]:
filename = f"{slug}.mdx"

blog_save_path = os.path.join(blog_path, filename)

with open(blog_save_path, 'w') as file:
    file.write(mdx_blog)

print(f"File '{filename}' created successfully.")

File 'the-moroccan-moussem-of-roses-valley-festival.mdx' created successfully.
